In [5]:
import os
from pathlib import Path
from tqdm import tqdm
import re
import pandas as pd
import numpy as np
from pandas import json_normalize
import time

from pydantic import BaseModel, Field
from typing import List, Optional, Sequence, Literal, Tuple

from IPython.display import Markdown, display
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [6]:
os.chdir("..")

In [2]:
from src.language_models.GPT import GPT

In [3]:
llm = GPT()

In [4]:
llm("hello world!")

'Hello! How can I assist you today?'

In [7]:
df = pd.read_parquet("data/reddit_legal_cluster_splade.parquet")

In [8]:
test_query = df['body'].head(1).tolist()[0]
print(test_query)

I applied for a job and after two interviews I was given for and agreed to an offer letter, had a start date.  The only remaining variable was a drug test, which I prepared myself for by bringing all my prescriptions with me.  

I take Adderall, my doctor prescribes me for 3 pills a day, insurance pays for only two a day so that’s what I get.  2 pills a day for 30 days.  

However, I don’t need adderall everyday, so I only take it as needed.  My psychiatrist knows this and is okay with it.  I don’t take it everyday so my prescription can last me months, depending on circumstances.  

Of course, I happen to take a drug test for a possible new job the day after taking an adderall at work.  No big deal, I have a prescription and it’s never been a big deal in the past.  

Except this time it was.  According to the doctor who administrated the test, my prescription was too old.  How old was it, I was prescribed in June and filled the prescription on July 8th, 2019.  I still have about a thi

In [9]:
class LegalIssue(BaseModel):
    """Identifying information about a legal issue."""
    topic: str = Field(..., description="A concise topic title for the legal issue")
    summary: Optional[str] = Field(None, description="A summary of key points")

class LegalIssues(BaseModel):
    """Identifying information in a text."""
    issues: Sequence[LegalIssue] = Field(..., description="The information of interest in the text")

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [15]:
test_llm = ChatOpenAI()

In [12]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world class algorithm for extracting information in structured formats.",
        ),
        (
            "human",
            "In an extremely legal tone, summarize from the following input: {input}",
        ),
        (
            "human", "Tip: Make sure to answer in the correct tone"),
    ]
)

In [13]:
prompt.format_messages(input=test_query)

[SystemMessage(content='You are a world class algorithm for extracting information in structured formats.'),
 HumanMessage(content='In an extremely legal tone, summarize from the following input: I applied for a job and after two interviews I was given for and agreed to an offer letter, had a start date.  The only remaining variable was a drug test, which I prepared myself for by bringing all my prescriptions with me.  \n\nI take Adderall, my doctor prescribes me for 3 pills a day, insurance pays for only two a day so that’s what I get.  2 pills a day for 30 days.  \n\nHowever, I don’t need adderall everyday, so I only take it as needed.  My psychiatrist knows this and is okay with it.  I don’t take it everyday so my prescription can last me months, depending on circumstances.  \n\nOf course, I happen to take a drug test for a possible new job the day after taking an adderall at work.  No big deal, I have a prescription and it’s never been a big deal in the past.  \n\nExcept this time 

In [18]:
test_res = test_llm.predict_messages(prompt.format_messages(input=test_query))

In [23]:
test_res

AIMessage(content='According to the information provided, the individual applied for a job and successfully went through the interview process. They were given an offer letter and had a set start date pending the completion of a drug test. The individual prepared for the drug test by bringing their prescription medications, specifically Adderall. Their doctor prescribed them three pills per day, but their insurance only covered two pills per day. They mentioned that they do not take Adderall every day and that their psychiatrist is aware and approves of this usage pattern. The individual had a drug test the day after taking Adderall, but the doctor administering the test claimed that their prescription was too old. The individual provided the date of their prescription fill, which was July 8th, 2019, and mentioned that they still had a third of a bottle remaining. The doctor refused to inspect the prescription bottle and demanded access to the individual\'s medical records. The individ